In [1]:
import ollama
import os
import pandas as pd

In [2]:
# warmup
# context 4K
# Internet Access NO
# System : Nvidia gpu
output = ollama.chat(
    model="llama3",
    messages=[
        {"role": "user", "content": "This is a test message, return a simple 1 line answer if you understand this message"}
    ]
)
outputs = []
output

ChatResponse(model='llama3', created_at='2025-10-08T20:08:43.2633508Z', done=True, done_reason='stop', total_duration=636285900, load_duration=97811800, prompt_eval_count=28, prompt_eval_duration=47622100, eval_count=7, eval_duration=490346500, message=Message(role='assistant', content='"I understand the test message!"', thinking=None, images=None, tool_name=None, tool_calls=None))

In [3]:
# TODO - Input Variations
# Different batch size inputs (1, 16, 128)
# Different Input tokens length (<20, <100)
# Different Complexity inputs (easy, hard)
# Different Output length (short, long)
# Different Systems (Mac, Windows)

In [4]:
def bench(prompts,prefix = "", stream = False):
    result = {}
    output = ollama.chat(
        model="llama3",
        messages=[
            {"role": "user", "content": prefix},
        ] + [{"role": "user", "content": prompt+"\n"} for prompt in prompts],
        stream = stream
    )

    result["prompts"] = len(prompts)
    result["input_tokens"] = output.prompt_eval_count
    result["output_tokens"] = output.eval_count
    result["total_time"] = output.total_duration/1e+6
    result["load_time"] = output.load_duration/1e+6
    result["prompt_eval_time"] = output.prompt_eval_duration/1e+6
    result["output_eval_time"] = output.eval_duration/1e+6

    return result, output.message.content

bench(["Do you know whos dashing, smart and disappears in a flash"])

({'prompts': 1,
  'input_tokens': 25,
  'output_tokens': 45,
  'total_time': 1205.0924,
  'load_time': 106.7844,
  'prompt_eval_time': 12.1352,
  'output_eval_time': 1083.8178},
 'I think I do! Is the answer "Zorro"? The legendary masked hero created by Johnston McCulley, known for his sword-fighting skills, cleverness, and ability to vanish quickly (in a flash!)')

In [ ]:
files = os.listdir("data/prompts/")
batch_sizes = [1,8,32,128]
device = "RTX3060"
for filename in files:
    _, questions, prompt_complexity, prompt_length = filename.split(".")[0].split("_")

    for batch_size in batch_sizes:
        with open(f"./data/prompts/{filename}", "r") as file:
            prompts = file.read().split("\\n")[:batch_size]

            for i in range(len(prompts)):
                prompts[i] = f"Question {i+1} : {prompts[i].strip()}"

        for output_length in ["short", "long"]:
            if output_length == "short":
                benchmark, answer = bench(prompts, prefix = f"give short answers for each of the following {len(prompts)} questions")
            else:
                benchmark, answer = bench(prompts, prefix = f"give long answers for each of the following {len(prompts)} questions")

            with open(f"./data/answers/answer_{batch_size}_{prompt_complexity}_{prompt_length}.txt", "w", encoding="utf-8") as file:
                file.write(answer)

            benchmark["prompt_complexity"] = prompt_complexity
            benchmark["prompt_length"] = prompt_length
            benchmark["output_length"] = output_length
            benchmark["device"] = device

            outputs.append(benchmark)
            outputs_df = pd.DataFrame(outputs)
            outputs_df.to_csv(f"./data/{device}_outputs.csv", index = False)

In [59]:
outputs_df = pd.DataFrame(outputs)
outputs_df.to_csv(f"./data/{device}outputs.csv", index = False)